# askEarth Explore API - Check Cloudcoverage

Before we get satellite data of an area of interest, we need to check which dates were available and how much each date was covered by clouds.
This notebook explains how to do just that.

## Step 1: Setup


In [2]:
import json
import requests


API_KEY = ""  # Paste the API key that you received from us here


def fetch_cloudcoverage(query):
    headers = {"Ocp-Apim-Subscription-Key": API_KEY, "Content-Type": "application/json"}
    data = json.dumps(query)
    return requests.post("https://ae-api.azure-api.net/askEarth/date-sc2a-cloudcover", data=data, headers=headers)

## Step 2: Prepare the query

The cloud coverage API requires us to specify an `area` and two `dates` (start and end).

The `area` is specified in the GeoJSON format. Learn more about it [here](https://geojson.org/).
It can be a hassle to write these by hand, so we recommend using a tool to make it easier such as [geojson.io](https://geojson.io). (Note: Make sure to only copy-paste the `geometry`.)

The `dates` object is a dictionary that has a `start` and `end` date in the format `yyyy-MM-dd`. The `start` date must come before the `end` date.

There is an optional `constellation` attribute that is `S2` by default (for Sentinel-2). You can specify it as `S1Land` for SAR.


In [5]:
query = {
    "area": {
        "coordinates": [
            [
                [8.472252130376006, 47.401720231335815],
                [8.472252130376006, 47.28481572263618],
                [8.622335960288467, 47.28481572263618],
                [8.622335960288467, 47.401720231335815],
                [8.472252130376006, 47.401720231335815],
            ]
        ],
        "type": "Polygon",
    },
    "dates": {"start": "2024-08-01", "end": "2024-08-18"},
    # "constellation": "S1Land"
}

## Step 3: Make the request

Now we will call the API and print the output.


In [6]:
response = fetch_cloudcoverage(query)

# Check if the response was successful
if response.status_code == 200:
    response_json = response.json()
    # Pretty print the response
    print(json.dumps(response_json, indent=2))
else:
    print("The response had an error with code:", response.status_code)
    print(response.content)

{
  "error": [
    {
      "type": "info",
      "status": 199,
      "message": "These (cloud) coverages are good estimates, but if your area of interest is small or thin, they may be off."
    }
  ],
  "result": [
    {
      "datetime": "2024-08-01T10:30:31.024000+00:00",
      "inverse_weighted_cloud_cover": 64.405394
    },
    {
      "datetime": "2024-08-03T10:15:59.024000+00:00",
      "inverse_weighted_cloud_cover": 59.063911
    },
    {
      "datetime": "2024-08-06T10:25:59.024000+00:00",
      "inverse_weighted_cloud_cover": 8.077753000000001
    },
    {
      "datetime": "2024-08-08T10:20:21.024000+00:00",
      "inverse_weighted_cloud_cover": 85.374713
    },
    {
      "datetime": "2024-08-11T10:30:31.024000+00:00",
      "inverse_weighted_cloud_cover": 15.488913999999994
    },
    {
      "datetime": "2024-08-13T10:15:59.024000+00:00",
      "inverse_weighted_cloud_cover": 24.383031000000003
    },
    {
      "datetime": "2024-08-16T10:25:59.024000+00:00",
      "i

## Step 4: Interpret the request

We received a list of dates with their respective cloud coverages. Let's get the date with the lowest cloud coverage.


In [8]:
def sort_by_cloud_cover(result):
    return sorted(result, key=lambda x: x["inverse_weighted_cloud_cover"])


sorted_dates = sort_by_cloud_cover(response_json["result"])

print(sorted_dates[0])

{'datetime': '2024-08-06T10:25:59.024000+00:00', 'inverse_weighted_cloud_cover': 8.077753000000001}


And this should give us the date `2024-08-06`.
